In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/sambodhan_classifier')  # adjust path to your project folder
!ls

requirements.txt  scripts


In [3]:
!pip install -r requirements.txt --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.0/98.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.7/459.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.33.2
    Uninstalling pydantic_core-2.33.2:
      Successfully uninstalled pydantic_core-2.33.2
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow

In [ ]:
# Upload your CSV file in Colab
from google.colab import files
uploaded = files.upload()

# Load data
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/sambodhan_classifier/sambodhan_balanced_dataset.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst 3 rows:")
print(df.head(3))

# Check class distribution
print("\n📊 Urgency Distribution:")
print(df['urgency'].value_counts())
print("\nPercentages:")
print(df['urgency'].value_counts(normalize=True) * 100)

# Check for missing values
print(f"\n Missing values:\n{df.isnull().sum()}")

# Sample texts from each class
for urgency_level in df['urgency'].unique():
    sample = df[df['urgency'] == urgency_level]['grievance'].iloc[0]
    print(f"\n{urgency_level} example: {sample[:100]}...")

Saving sambodhan_balanced_dataset.csv to sambodhan_balanced_dataset.csv
Dataset shape: (6000, 4)

Columns: ['id', 'urgency', 'grievance', 'department']

First 3 rows:
      id        urgency                                          grievance  \
0  83223  HIGHLY URGENT  ओम शिव शक्ति बचत तथा ऋण सहकारी संस्थाले युवा स...   
1  83224         URGENT  यी हाम्रा मुल सिमा खम्बा जंगे पिलरहरुले नेपाल ...   
2  83225         URGENT  भक्तपुर मनोहरा फाट बाट नरेफाट तिर जाने ,बिपि न...   

                                      department  
0  Infrastructure, Utilities & Natural Resources  
1                     Security & Law Enforcement  
2                     Security & Law Enforcement  

📊 Urgency Distribution:
urgency
URGENT           2438
NORMAL           1836
HIGHLY URGENT    1726
Name: count, dtype: int64

Percentages:
urgency
URGENT           40.633333
NORMAL           30.600000
HIGHLY URGENT    28.766667
Name: proportion, dtype: float64

❌ Missing values:
id            0
urgency       0
grie

## Data preparation

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/sambodhan_classifier/scripts')
from data_prep import load_and_prepare_data

# Upload your CSV dataset if not already in Drive
from google.colab import files
uploaded = files.upload()
csv_file = list(uploaded.keys())[0]

train_df, val_df, test_df = load_and_prepare_data(csv_file)

Saving sambodhan_balanced_dataset.csv to sambodhan_balanced_dataset.csv
Loaded 6000 rows
Train: 4200, Val: 900, Test: 900


In [6]:
import sys
sys.path.append('/content/drive/MyDrive/sambodhan_classifier/scripts')
from train import train_model

from google.colab import files
uploaded = files.upload()
csv_file = list(uploaded.keys())[0]

# Start training
model, tokenizer, trainer = train_model(
        csv_file=csv_file,
        output_dir="/content/drive/MyDrive/sambodhan_classifier/sambodhan_model_final",
        epochs=15,
        batch_size=16,
        gradient_accumulation_steps=2,
        learning_rate=3e-5,
        max_length=96,
        weight_decay=0.02,
        dropout=0.3,
        use_focal_loss=True,
        focal_gamma=2.0,
        augment_multiplier=1.5,
        freeze_embeddings=False,
        cleanup_after_training=True
    )

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving sambodhan_balanced_dataset.csv to sambodhan_balanced_dataset (2).csv
Loaded 6000 rows
Train: 4200, Val: 900, Test: 900
Original train size: 4200
Class distribution:
label
1    1707
0    1285
2    1208
Name: count, dtype: int64

After balanced augmentation: 5781 samples
New class distribution:
label
0    1927
2    1927
1    1927
Name: count, dtype: int64


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bistakaran89 (bistakaran89-fusemachines) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted,F1 Class 0,Precision Class 0,Recall Class 0,F1 Class 1,Precision Class 1,Recall Class 1,F1 Class 2,Precision Class 2,Recall Class 2
1,0.614600,0.502293,0.516667,0.473817,0.442306,0.622061,0.618705,0.625455,0.186603,0.750000,0.106557,0.612786,0.445614,0.980695
2,0.288300,0.493442,0.687778,0.673592,0.651850,0.825503,0.766355,0.894545,0.468000,0.873134,0.319672,0.727273,0.575281,0.988417
3,0.129400,0.162652,0.855556,0.859620,0.855845,0.894212,0.991150,0.814545,0.822695,0.855457,0.792350,0.861953,0.764179,0.988417
4,0.099400,0.130960,0.917778,0.919725,0.917013,0.908772,0.877966,0.941818,0.898551,0.956790,0.846995,0.951852,0.914591,0.992278
5,0.077900,0.170674,0.916667,0.917945,0.916332,0.916667,0.956522,0.880000,0.904564,0.915966,0.893443,0.932605,0.882759,0.988417
6,0.084200,0.164308,0.925556,0.926333,0.925092,0.909449,0.991416,0.840000,0.918421,0.885787,0.953552,0.951128,0.926740,0.976834
7,0.054800,0.114647,0.940000,0.941275,0.939789,0.920518,0.936090,0.905455,0.931880,0.929348,0.934426,0.971429,0.958647,0.984556
8,0.051000,0.110709,0.938889,0.941090,0.938795,0.917127,0.929104,0.905455,0.925373,0.919137,0.931694,0.980769,0.977011,0.984556
9,0.047200,0.087112,0.941111,0.942811,0.940930,0.921101,0.929630,0.912727,0.930233,0.931507,0.928962,0.977099,0.966038,0.988417
10,0.045800,0.114042,0.945556,0.946880,0.945240,0.918519,0.935849,0.901818,0.937330,0.934783,0.939891,0.984791,0.970037,1.000000



📊 Final validation metrics:


eval_loss: 0.1415
eval_accuracy: 0.9467
eval_f1_macro: 0.9480
eval_f1_weighted: 0.9464
eval_f1_class_0: 0.9245
eval_precision_class_0: 0.9366
eval_recall_class_0: 0.9127
eval_f1_class_1: 0.9384
eval_precision_class_1: 0.9397
eval_recall_class_1: 0.9372
eval_f1_class_2: 0.9810
eval_precision_class_2: 0.9663
eval_recall_class_2: 0.9961
epoch: 15.0000

✅ Loading best checkpoint from: /content/drive/MyDrive/sambodhan_classifier/sambodhan_model_final/checkpoint-2353

💾 Saving final model to: /content/drive/MyDrive/sambodhan_classifier/sambodhan_model_final

📋 DETAILED CLASSIFICATION REPORT
               precision    recall  f1-score   support

       NORMAL     0.9366    0.9127    0.9245       275
       URGENT     0.9397    0.9372    0.9384       366
HIGHLY URGENT     0.9663    0.9961    0.9810       259

     accuracy                         0.9467       900
    macro avg     0.9475    0.9487    0.9480       900
 weighted avg     0.9464    0.9467    0.9464       900

Confusion matrix sav

In [3]:
# === Colab Runner for Sambodhan Evaluation ===

import sys
sys.path.append('/content/drive/MyDrive/sambodhan_classifier/scripts')
import pandas as pd
from eval import evaluate_model  # Make sure eval.py is in the same folder or in sys.path

# Ensure the test dataset has required columns
if not set(["clean_text", "label"]).issubset(test_df.columns):
    raise ValueError("Test CSV must contain 'clean_text' and 'label' columns.")

print(f"✅ Loaded test dataset with {len(test_df)} samples.")

# --- RUN EVALUATION ---
results = evaluate_model(
   test_df=test_df,
  )

# --- SUMMARY OUTPUT ---
print("\n=== Evaluation Summary ===")
print(results["summary"])
print(f"TensorBoard run directory: {results['run_dir']}")
print(f"All plots, CSVs, and reports saved in: {results['run_dir']}")

✅ Loaded test dataset with 900 samples.
🔍 Loading model from: /content/drive/MyDrive/sambodhan_classifier/sambodhan_model_final


Device set to use cuda:0


⚙️ Evaluating on 900 samples using GPU...


Inference: 100%|██████████████████████████████████████████████████| 900/900 [00:11<00:00, 80.16it/s]


✅ Evaluation complete in 33.70s.
📤 Uploading TensorBoard logs to Hugging Face...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...7267.8eafa203e83b.11024.0: 100%|##########| 1.78kB / 1.78kB            

✅ Uploaded run to https://huggingface.co/kar137/sambodhan-urgency-classifier/tensorboard

=== Evaluation Summary ===
📊 EVALUATION SUMMARY
Dataset: Sambodhan Urgency Dataset
Samples: 900
Accuracy: 0.9344
Macro F1: 0.9362
Weighted F1: 0.9343
Misclassified Samples: 59
Run: Oct10_14-41-04
TensorBoard run directory: /content/drive/MyDrive/sambodhan_classifier/evaluation_outputs/runs/Oct10_14-41-04
All plots, CSVs, and reports saved in: /content/drive/MyDrive/sambodhan_classifier/evaluation_outputs/runs/Oct10_14-41-04


In [ ]:
!pip install -U huggingface_hub

In [ ]:
from huggingface_hub import login, HfApi
import os, json, getpass

# ===== SECURE: Load token from environment or prompt =====
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
if not HF_TOKEN:
    try:
        HF_TOKEN = getpass.getpass("Enter your Hugging Face token (input hidden): ")
    except Exception:
        HF_TOKEN = None

if not HF_TOKEN:
    print("WARNING: No Hugging Face token provided. Some steps will be skipped.")
else:
    # Login (stores token in runtime)
    login(token=HF_TOKEN)
    
    # Initialize API client and get username
    api = HfApi()
    user = api.whoami(token=HF_TOKEN)
    username = user.get("name") or user.get("id") or user.get("login")
    print("Logged in as:", username)

# ===== EDIT THESE PATHS if your files are in different places =====
# Project root in Colab (where your repository lives or where you uploaded files)
PROJECT_ROOT = "/content/drive/MyDrive/sambodhan_classifier"   # change if needed
MODEL_DIR = "/content/drive/MyDrive/sambodhan_classifier/sambodhan_model_final"          # change if needed (path to trained model folder)
SPACE_APP_DIR = "/content/sambodhan_space"      # temporary folder for Space files
os.makedirs(SPACE_APP_DIR, exist_ok=True)

# Quick check - list files
print("Project root exists?", os.path.exists(PROJECT_ROOT))
print("Model dir exists?", os.path.exists(MODEL_DIR))
!ls -la

Logged in as: kar137
Project root exists? True
Model dir exists? True
total 4688
drwx------ 3 root root    4096 Oct 10 06:41  evaluation_outputs
-rw------- 1 root root     107 Oct 10 04:00  requirements.txt
-rw------- 1 root root 1594082 Oct 10 06:19 'sambodhan_balanced_dataset (1).csv'
-rw------- 1 root root 1594082 Oct 10 06:20 'sambodhan_balanced_dataset (2).csv'
-rw------- 1 root root 1594082 Oct 10 06:16  sambodhan_balanced_dataset.csv
drwx------ 2 root root    4096 Oct 10 06:41  sambodhan_model_final
drwx------ 3 root root    4096 Oct 10 03:56  scripts
drwx------ 3 root root    4096 Oct 10 06:21  wandb


In [ ]:
import shutil, os

output_dir = "/content/drive/MyDrive/sambodhan_classifier/sambodhan_model_final"

for folder in os.listdir(output_dir):
    if folder.startswith("checkpoint") or folder == "logs":
        shutil.rmtree(os.path.join(output_dir, folder))
print("✅ Cleaned up checkpoints and logs, keeping only the final model files.")

✅ Cleaned up checkpoints and logs, keeping only the final model files.


In [ ]:
from huggingface_hub import HfApi, upload_folder

api = HfApi()

# repo id you will create (change name if you want)
model_repo_id = f"kar137/sambodhan-urgency-classifier"

# Create model repo (exist_ok=True tolerates existing repo)
api.create_repo(repo_id=model_repo_id, repo_type="model", exist_ok=True, token=HF_TOKEN)

print("Uploading model folder to:", model_repo_id)
# Upload the entire folder; this uses the HF API (no git needed)
upload_folder(
    folder_path=MODEL_DIR,
    repo_id=model_repo_id,
    repo_type="model",
    token=HF_TOKEN,
    ignore_patterns=["*.pyc","__pycache__"]
)

print("Upload finished. Model is at:", f"https://huggingface.co/{model_repo_id}")

Uploading model folder to: kar137/sambodhan-urgency-classifier


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...l/sentencepiece.bpe.model: 100%|##########| 5.07MB / 5.07MB            

  ...l_final/model.safetensors:   0%|          | 13.5kB / 1.11GB            

  ...l_final/training_args.bin:  10%|9         |   583B / 5.91kB            

Upload finished. Model is at: https://huggingface.co/kar137/sambodhan-urgency-classifier


In [ ]:
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1
print("Using device:", device)

# Load the pipeline from the local model directory
classifier = pipeline("text-classification", model=MODEL_DIR, device=device)
print(classifier("पानीको समस्या धेरै बढेको छ"))

Using device: 0


Device set to use cuda:0


[{'label': 'HIGHLY URGENT', 'score': 0.8974823951721191}]


In [ ]:
#  SETUP
from huggingface_hub import HfApi, upload_folder
import os, getpass

# Load token securely
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
if not HF_TOKEN:
    try:
        HF_TOKEN = getpass.getpass("Enter your Hugging Face token (input hidden): ")
    except Exception:
        HF_TOKEN = None

if not HF_TOKEN:
    print("WARNING: No token provided. Space upload will be skipped.")
else:
    # Model & Space repos
    model_repo_id = "kar137/sambodhan-urgency-classifier"
    space_repo_id = "kar137/sambodhan-urgency-classifier-space"
    
    # Local Space folder
    SPACE_APP_DIR = "./sambodhan_fastapi_space"
    os.makedirs(SPACE_APP_DIR, exist_ok=True)
    
    
    # 1️. app.py (FastAPI + cache + safe dependencies)
    
    app_py = f"""
import os
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

# ✅ Create writable model cache directory
os.makedirs("/app/model_cache", exist_ok=True)
os.environ["HF_HOME"] = "/app/model_cache"
os.environ["TRANSFORMERS_CACHE"] = "/app/model_cache"
os.environ["HF_DATASETS_CACHE"] = "/app/model_cache"
os.environ["HF_METRICS_CACHE"] = "/app/model_cache"

MODEL_REPO = "{model_repo_id}"
device = 0 if torch.cuda.is_available() else -1

# ✅ Load model and tokenizer safely
tokenizer = AutoTokenizer.from_pretrained(MODEL_REPO, cache_dir="/app/model_cache")
model = AutoModelForSequenceClassification.from_pretrained(MODEL_REPO, cache_dir="/app/model_cache")

classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=device,
    return_all_scores=True
)

app = FastAPI(title="Sambodhan Urgency Classifier")

class PredictRequest(BaseModel):
    text: str

class PredictResponse(BaseModel):
    label: str
    confidence: float
    all_scores: dict

@app.get("/")
def root():
    return {{"message": "Sambodhan Urgency Classifier is running!"}}

@app.post("/predict", response_model=PredictResponse)
def predict(req: PredictRequest):
    results = classifier(req.text)[0]
    top = max(results, key=lambda x: x['score'])
    return PredictResponse(
        label=top['label'],
        confidence=round(top['score'], 3),
        all_scores={{r['label']: round(r['score'], 3) for r in results}}
    )
"""
    
    # Write app.py
    with open(f"{{SPACE_APP_DIR}}/app.py", "w", encoding="utf-8") as f:
        f.write(app_py)
    
    # 2️. requirements.txt
    reqs = """fastapi
uvicorn[standard]
transformers
torch
pydantic
"""
    with open(f"{{SPACE_APP_DIR}}/requirements.txt", "w", encoding="utf-8") as f:
        f.write(reqs)
    
    # 3️. Dockerfile
    dockerfile = """FROM python:3.10-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY app.py .

EXPOSE 7860

CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "7860"]
"""
    with open(f"{{SPACE_APP_DIR}}/Dockerfile", "w", encoding="utf-8") as f:
        f.write(dockerfile)
    
    # 4️. README.md for Space
    readme = f"""---
title: Sambodhan Urgency Classifier
emoji: 🚨
colorFrom: red
colorTo: orange
sdk: docker
app_port: 7860
---

# Sambodhan Urgency Classifier FastAPI

This Space hosts the urgency classification model for Sambodhan grievances.

## Endpoint

- **POST /predict**

Example:
```bash
curl -X POST "https://{{space_repo_id.replace('/', '-')}}.hf.space/predict" \\
  -H "Content-Type: application/json" \\
  -d '{{"text": "मलाई तुरुन्त सहयोग चाहिन्छ"}}'
```
"""
    with open(f"{{SPACE_APP_DIR}}/README.md", "w", encoding="utf-8") as f:
        f.write(readme)
    
    print(f"✅ Space files ready in {{SPACE_APP_DIR}}")
    print("Contents:")
    !ls -la {{SPACE_APP_DIR}}

✅ Docker Space created & uploaded successfully!
🌐 Space URL: https://huggingface.co/spaces/kar137/sambodhan-urgency-classifier-space
🚀 Endpoint: https://kar137-sambodhan-urgency-classifier-space.hf.space/predict


In [ ]:
import requests

url = "https://kar137-sambodhan-urgency-classifier-space.hf.space/predict"
data = {"text": "मलाई तुरुन्त सहयोग चाहिन्छ"}
print(requests.post(url, json=data).json())

{'label': 'HIGHLY URGENT', 'confidence': 0.9861, 'scores': {'NORMAL': 0.002, 'URGENT': 0.0119, 'HIGHLY URGENT': 0.9861}}


In [ ]:
app_py = f'''
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

MODEL_REPO = "{model_repo_id}"

device = 0 if torch.cuda.is_available() else -1

# Load model & tokenizer from Hub
tokenizer = AutoTokenizer.from_pretrained(MODEL_REPO)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_REPO)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device, return_all_scores=True)

LABELS = ["NORMAL","URGENT","HIGHLY URGENT"]  # change if your label mapping differs

def predict(text):
    if not text or not text.strip():
        return {{ "error": "Empty input" }}
    results = classifier(text)[0]
    top = max(results, key=lambda x: x["score"])
    return {{
        "label": top["label"],
        "confidence": round(top["score"], 4),
        "scores": {{ r["label"]: round(r["score"], 4) for r in results }}
    }}

iface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=3, placeholder="Enter grievance text in Nepali/English..."),
    outputs="json",
    title="Sambodhan Urgency Classifier",
    description="Loads model from Hugging Face Hub and returns urgency + confidence."
)

if __name__ == "__main__":
    iface.launch()
'''

with open(os.path.join(SPACE_APP_DIR, "app.py"), "w", encoding="utf-8") as f:
    f.write(app_py)

# Minimal requirements for the Space
reqs = """transformers
torch
gradio
huggingface-hub
"""
with open(os.path.join(SPACE_APP_DIR, "requirements.txt"), "w") as f:
    f.write(reqs)

!ls -la {SPACE_APP_DIR}


total 16
drwxr-xr-x 2 root root 4096 Oct  9 10:54 .
drwxr-xr-x 1 root root 4096 Oct  9 10:37 ..
-rw-r--r-- 1 root root 1259 Oct  9 10:54 app.py
-rw-r--r-- 1 root root   42 Oct  9 10:54 requirements.txt


In [ ]:
from huggingface_hub import HfApi, upload_folder

api = HfApi()

# repo id you will create (change name if you want)
space_repo_id = f"kar137/sambodhan-urgency-classifier-space"

# Create Space repo
api.create_repo(repo_id=space_repo_id, repo_type="space", exist_ok=True, token=HF_TOKEN, space_sdk="gradio")

# Upload the small app folder to the Space repo
upload_folder(
    folder_path=SPACE_APP_DIR,
    repo_id=space_repo_id,
    repo_type="space",
    token=HF_TOKEN,
    ignore_patterns=["*.pyc","__pycache__"]
)

print("Space created. Open:", f"https://huggingface.co/spaces/{space_repo_id}")

Space created. Open: https://huggingface.co/spaces/kar137/sambodhan-urgency-classifier-space
